# 🤟 Sign Language Translator - VERSIÓN COMPLETA

## ✨ Características Completas
- 📹 **Webcam en Gradio** (dentro de la interfaz)
- 🎥 **Captura múltiple** con detección de manos
- 🎓 **Entrenamiento** con Bi-LSTM + Attention
- 🔮 **Traducción** en tiempo real
- 📊 **240 Features** por frame
- 🔄 **Data Augmentation**

---

## 📋 Flujo Completo

### 1. Captura (Tab 1)
- Webcam en vivo en Gradio
- Click "Preview" → Ver mano + keypoints
- Click "Capturar" → Graba múltiples muestras
- Auto-pausa sin manos

### 2. Entrenamiento (Tab 2)
- Carga datos capturados
- Entrena Bi-LSTM + Attention
- Data augmentation automático
- Guarda mejor modelo

### 3. Traducción (Tab 3)
- Carga modelo entrenado
- Webcam en tiempo real
- Traduce señas instantáneamente
- Muestra confianza

### 4. Dataset Info (Tab 4)
- Estadísticas de datos
- Muestras por seña
- Recomendaciones

---
# 🔧 CELDA 1: Setup

In [ ]:
print("="*70)
print("🤟 Sign Language Translator - Setup")
print("="*70)

# 1. Montar Drive
print("\n📁 Paso 1/6: Montando Google Drive...")
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
print("✅ Drive montado")

# 2. Clonar repo
print("\n📥 Paso 2/6: Clonando repositorio...")
import os
os.chdir('/content')

GITHUB_REPO = "https://github.com/Hernan-Mej/sign-language-translator.git"

if os.path.exists('sign-language-translator'):
    !cd sign-language-translator && git pull
else:
    !git clone {GITHUB_REPO}
print("✅ Repositorio listo")

# 3. Instalar deps
print("\n📦 Paso 3/6: Instalando dependencias...")
!pip install -q protobuf==4.25.8
!pip install -q -r /content/sign-language-translator/requirements.txt
print("✅ Dependencias instaladas")

# 4. Paths
print("\n🔧 Paso 4/6: Configurando paths...")
import sys
sys.path.insert(0, '/content/sign-language-translator/src')
print("✅ Paths configurados")

# 5. Estructura Drive
print("\n📂 Paso 5/6: Creando estructura...")
from pathlib import Path

drive_root = Path('/content/drive/MyDrive')
project_dir = drive_root / 'SignLanguageTranslator'

for directory in [
    project_dir / 'data' / 'raw',
    project_dir / 'data' / 'processed',
    project_dir / 'models',
    project_dir / 'logs',
]:
    directory.mkdir(parents=True, exist_ok=True)

print("✅ Estructura creada")

# 6. Verificar imports
print("\n🧪 Paso 6/6: Verificando módulos...")
try:
    from config import MODEL_CONFIG, DATA_DIR, MODELS_DIR
    from key_points_extractor import KeyPointsExtractor
    from data_augmentation import SignLanguageAugmenter
    print("✅ Módulos básicos OK")
    
    # Intentar importar training
    try:
        from advanced_lstm_model import create_model
        from training import Trainer
        print("✅ Módulos de entrenamiento OK")
    except ImportError:
        print("⚠️  Módulos de entrenamiento no encontrados")
        print("   Para entrenar, agrega advanced_lstm_model.py y training.py")
except ImportError as e:
    print(f"❌ Error: {e}")

print("\n" + "="*70)
print("✅ SETUP COMPLETADO")
print("="*70)
print("\n📌 Ejecuta la celda de UI (abajo)")

---
# 🎨 CELDA 2: Interfaz Completa

In [ ]:
# ============================================================================
# INTERFAZ COMPLETA CON WEBCAM EN GRADIO
# ============================================================================

import gradio as gr
import cv2
import numpy as np
import json
import time
from pathlib import Path
from datetime import datetime
import tensorflow as tf

# Imports del proyecto
from config import DATA_DIR, MODELS_DIR, MODEL_CONFIG
from key_points_extractor import KeyPointsExtractor
from data_augmentation import SignLanguageAugmenter

# Intentar importar training
try:
    from advanced_lstm_model import create_model
    from training import Trainer
    TRAINING_AVAILABLE = True
except ImportError:
    TRAINING_AVAILABLE = False
    print("⚠️  Módulos de entrenamiento no disponibles")

# ============================================================================
# ESTADO GLOBAL
# ============================================================================

class AppState:
    def __init__(self):
        self.extractor = KeyPointsExtractor()
        self.augmenter = SignLanguageAugmenter()
        self.current_model = None
        self.sign_map = {}
        self.capturing = False
        self.capture_buffer = []

state = AppState()

# ============================================================================
# FUNCIONES DE PROCESAMIENTO
# ============================================================================

def process_frame(frame):
    """Procesa frame con MediaPipe y extrae features."""
    if frame is None:
        return None, np.zeros(240), False
    
    # Convertir RGB a BGR para OpenCV
    frame_bgr = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
    
    # Extraer keypoints
    processed_frame, features = state.extractor.extract_keypoints(frame_bgr)
    
    # Detectar si hay mano
    hand_detected = not np.all(features == 0)
    
    # Convertir de vuelta a RGB para Gradio
    processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_BGR2RGB)
    
    return processed_frame, features, hand_detected

# ============================================================================
# TAB 1: CAPTURA
# ============================================================================

def preview_webcam(frame):
    """Preview de webcam con detección."""
    if frame is None:
        return None, "⚠️ No hay frame de webcam"
    
    processed_frame, features, hand_detected = process_frame(frame)
    
    status = "✅ Mano detectada" if hand_detected else "⚠️ No se detecta mano"
    non_zero = np.count_nonzero(features)
    info = f"{status}\n📊 Features: {non_zero}/240"
    
    return processed_frame, info

def start_capture(sign_name, num_samples, frames_per_sample):
    """Inicia captura múltiple."""
    if not sign_name or not sign_name.strip():
        return "❌ Ingresa un nombre para la seña", gr.update(interactive=False)
    
    state.capturing = True
    state.capture_buffer = []
    state.current_sign = sign_name.strip().lower().replace(" ", "_")
    state.total_samples = num_samples
    state.frames_per_sample = frames_per_sample
    state.samples_captured = 0
    state.current_sequence = []
    
    return f"✅ Iniciando captura de '{state.current_sign}'\n🎯 Meta: {num_samples} muestras\n\n▶️ Muestra tu mano para empezar...", gr.update(interactive=True)

def capture_frame(frame):
    """Captura frame durante sesión de captura."""
    if not state.capturing:
        return frame, "⏹️ No hay captura activa"
    
    if frame is None:
        return None, "⚠️ No hay frame"
    
    processed_frame, features, hand_detected = process_frame(frame)
    
    status = f"📹 Capturando: {state.current_sign}\n"
    status += f"📊 Muestra {state.samples_captured + 1}/{state.total_samples}\n"
    status += f"📐 Frames: {len(state.current_sequence)}/{state.frames_per_sample}\n\n"
    
    if hand_detected:
        state.current_sequence.append(features)
        status += "✅ Grabando...\n"
        
        # Completar muestra
        if len(state.current_sequence) >= state.frames_per_sample:
            # Guardar muestra
            sign_dir = DATA_DIR / "raw" / state.current_sign
            sign_dir.mkdir(parents=True, exist_ok=True)
            
            existing = len(list(sign_dir.glob("*.npy")))
            sequence_array = np.array(state.current_sequence[:state.frames_per_sample])
            
            sample_file = sign_dir / f"sample_{existing:03d}.npy"
            np.save(sample_file, sequence_array)
            
            state.samples_captured += 1
            state.current_sequence = []
            
            status += f"✅ Muestra {state.samples_captured} guardada!\n"
            
            # Completar todas las muestras
            if state.samples_captured >= state.total_samples:
                state.capturing = False
                status += f"\n🎉 COMPLETADO!\n"
                status += f"📦 {state.samples_captured} muestras guardadas\n"
                status += f"📁 {sign_dir}\n"
            else:
                status += f"\n⏸️ Pausa 1 seg - Cambia ángulo\n"
                time.sleep(1)
    else:
        status += "⏸️ Esperando mano...\n"
    
    return processed_frame, status

def stop_capture():
    """Detiene captura."""
    state.capturing = False
    return f"⏹️ Captura detenida\n📊 Muestras capturadas: {state.samples_captured}/{state.total_samples}"

# ============================================================================
# TAB 2: ENTRENAMIENTO
# ============================================================================

def train_model(epochs, batch_size, use_augmentation, progress=gr.Progress()):
    """Entrena el modelo."""
    if not TRAINING_AVAILABLE:
        return "❌ Módulos de entrenamiento no disponibles\n\nAgrega estos archivos a tu repo:\n- advanced_lstm_model.py\n- training.py\n\nCódigo en: ALL_FILES_COMPLETE_CODE.md"
    
    try:
        progress(0, desc="Inicializando...")
        
        trainer = Trainer(
            data_dir=DATA_DIR,
            models_dir=MODELS_DIR,
            use_augmentation=use_augmentation
        )
        
        progress(0.1, desc="Cargando datos...")
        X, y, sign_map = trainer.load_and_prepare_data()
        
        if len(X) == 0:
            return "❌ No hay datos. Captura algunas señas primero."
        
        state.sign_map = sign_map
        
        progress(0.2, desc="Normalizando...")
        X = trainer.normalize_sequence_lengths(X, target_length=30)
        
        progress(0.3, desc="Dividiendo datos...")
        X_train, X_val, X_test, y_train, y_val, y_test = trainer.split_data(X, y)
        
        if use_augmentation:
            progress(0.4, desc="Data augmentation...")
            X_train, y_train = trainer.augment_data(X_train, y_train)
        
        progress(0.5, desc="Entrenando modelo...")
        model, history = trainer.train_model(
            X_train, y_train,
            X_val, y_val,
            num_classes=len(sign_map),
            epochs=epochs,
            batch_size=batch_size,
            model_name="sign_language_model"
        )
        
        state.current_model = model
        
        progress(0.9, desc="Evaluando...")
        results = trainer.evaluate_model(model, X_test, y_test, sign_map)
        
        progress(1.0, desc="Completado")
        
        output = f"✅ ENTRENAMIENTO COMPLETADO\n\n"
        output += f"📊 Dataset:\n"
        output += f"   Señas: {len(sign_map)}\n"
        output += f"   Train: {len(X_train)}\n"
        output += f"   Val: {len(X_val)}\n"
        output += f"   Test: {len(X_test)}\n\n"
        output += f"📈 Resultados:\n"
        output += f"   Accuracy: {results['metrics']['accuracy']:.2%}\n"
        output += f"   Top-3: {results['metrics'].get('top_3_accuracy', 0):.2%}\n\n"
        output += f"💾 Modelo: sign_language_model_best.h5\n"
        
        return output
        
    except Exception as e:
        return f"❌ Error: {str(e)}"

# ============================================================================
# TAB 3: TRADUCCIÓN
# ============================================================================

def load_model_for_translation(model_name):
    """Carga modelo para traducción."""
    try:
        model_path = MODELS_DIR / model_name
        if not model_path.exists():
            return "❌ Modelo no encontrado"
        
        state.current_model = tf.keras.models.load_model(str(model_path))
        
        # Cargar sign_map
        sign_map_path = DATA_DIR / "processed" / "sign_map.json"
        if sign_map_path.exists():
            with open(sign_map_path) as f:
                state.sign_map = {int(k): v for k, v in json.load(f).items()}
        
        return f"✅ Modelo cargado: {model_name}"
    except Exception as e:
        return f"❌ Error: {str(e)}"

def translate_sign(frame):
    """Traduce seña en tiempo real."""
    if state.current_model is None:
        return frame, "⚠️ Carga un modelo primero"
    
    if frame is None:
        return None, "⚠️ No hay frame"
    
    # Procesar frame
    processed_frame, features, hand_detected = process_frame(frame)
    
    if not hand_detected:
        return processed_frame, "⏸️ No se detecta mano"
    
    # Acumular frames (necesitamos secuencia de 30)
    if not hasattr(state, 'translation_buffer'):
        state.translation_buffer = []
    
    state.translation_buffer.append(features)
    
    if len(state.translation_buffer) > 30:
        state.translation_buffer.pop(0)
    
    if len(state.translation_buffer) < 30:
        return processed_frame, f"📊 Acumulando frames: {len(state.translation_buffer)}/30"
    
    # Predecir
    sequence = np.array([state.translation_buffer])
    prediction = state.current_model.predict(sequence, verbose=0)
    predicted_class = np.argmax(prediction[0])
    confidence = prediction[0][predicted_class]
    
    # Obtener nombre
    sign_name = state.sign_map.get(predicted_class, f"Clase {predicted_class}")
    
    result = f"🔮 Seña: {sign_name.upper()}\n"
    result += f"📊 Confianza: {confidence:.1%}\n\n"
    
    # Top 3
    top3_idx = np.argsort(prediction[0])[-3:][::-1]
    result += "Top 3:\n"
    for i, idx in enumerate(top3_idx, 1):
        name = state.sign_map.get(idx, f"Clase {idx}")
        conf = prediction[0][idx]
        result += f"  {i}. {name}: {conf:.1%}\n"
    
    return processed_frame, result

# ============================================================================
# TAB 4: DATASET INFO
# ============================================================================

def show_dataset_info():
    """Muestra info del dataset."""
    raw_dir = DATA_DIR / "raw"
    if not raw_dir.exists():
        return "📭 No hay datos capturados"
    
    sign_dirs = [d for d in raw_dir.iterdir() if d.is_dir()]
    
    if not sign_dirs:
        return "📭 No hay señas capturadas"
    
    output = f"📊 DATASET DE SEÑAS\n" + "="*50 + "\n\n"
    output += f"🎯 Total de señas: {len(sign_dirs)}\n\n"
    
    total_samples = 0
    
    for sign_dir in sorted(sign_dirs):
        samples = list(sign_dir.glob("*.npy"))
        num_samples = len(samples)
        total_samples += num_samples
        
        output += f"📁 {sign_dir.name}:\n"
        output += f"   Muestras: {num_samples}\n"
        
        if samples:
            data = np.load(samples[0])
            output += f"   Shape: {data.shape}\n"
        
        if num_samples < 10:
            output += f"   ⚠️  Mínimo recomendado: 15\n"
        elif num_samples >= 15:
            output += f"   ✅ Excelente\n"
        else:
            output += f"   ✅ Aceptable\n"
        
        output += "\n"
    
    output += "="*50 + "\n"
    output += f"📈 Total muestras: {total_samples}\n"
    output += f"📊 Promedio/seña: {total_samples / len(sign_dirs):.1f}\n\n"
    
    # Recomendaciones
    if len(sign_dirs) < 5:
        output += "💡 Captura al menos 5 señas diferentes\n"
    if total_samples < 75:
        output += "💡 Necesitas ~75 muestras (15 por seña × 5 señas)\n"
    
    return output

def get_model_list():
    """Lista modelos disponibles."""
    models = [m.name for m in MODELS_DIR.glob("*.h5")]
    return models if models else ["No hay modelos"]

# ============================================================================
# CREAR INTERFAZ
# ============================================================================

with gr.Blocks(theme=gr.themes.Soft(), title="Sign Language Translator") as demo:
    
    gr.Markdown("""
    # 🤟 Sign Language Translator - COMPLETO
    ### ✨ Captura • Entrenamiento • Traducción
    
    **Webcam integrada en Gradio • 240 features • Bi-LSTM + Attention • Data Augmentation**
    """)
    
    with gr.Tabs():
        
        # ====================================================================
        # TAB 1: CAPTURA
        # ====================================================================
        
        with gr.Tab("📹 Captura"):
            gr.Markdown("""
            ### Captura múltiple con detección de manos
            
            **Flujo:**
            1. Autoriza webcam en tu navegador
            2. Click "Preview" para verificar detección
            3. Ingresa nombre de seña
            4. Click "Iniciar Captura"
            5. Muestra tu mano → Graba automáticamente
            6. Captura múltiples muestras variando ángulos
            """)
            
            with gr.Row():
                with gr.Column(scale=1):
                    webcam_input = gr.Image(sources=["webcam"], streaming=True, label="Webcam")
                
                with gr.Column(scale=1):
                    webcam_output = gr.Image(label="Procesado + Keypoints")
                    status_output = gr.Textbox(label="Estado", lines=8)
            
            with gr.Row():
                sign_name_input = gr.Textbox(label="Nombre de la Seña", placeholder="Ej: hola")
                num_samples_input = gr.Slider(1, 30, 15, step=1, label="Muestras")
                frames_input = gr.Slider(15, 45, 30, step=5, label="Frames/Muestra")
            
            with gr.Row():
                preview_btn = gr.Button("👁️ Preview", variant="secondary")
                start_btn = gr.Button("▶️ Iniciar Captura", variant="primary")
                stop_btn = gr.Button("⏹️ Detener", variant="stop")
            
            # Eventos
            preview_btn.click(
                fn=preview_webcam,
                inputs=webcam_input,
                outputs=[webcam_output, status_output]
            )
            
            start_btn.click(
                fn=start_capture,
                inputs=[sign_name_input, num_samples_input, frames_input],
                outputs=[status_output, start_btn]
            )
            
            # Stream processing durante captura
            webcam_input.stream(
                fn=capture_frame,
                inputs=webcam_input,
                outputs=[webcam_output, status_output],
                stream_every=0.1
            )
            
            stop_btn.click(
                fn=stop_capture,
                outputs=status_output
            )
            
            gr.Markdown("""
            **💡 Tips:**
            - Captura 15-20 muestras por seña
            - Varía ángulos entre muestras
            - Auto-pausa cuando no detecta manos
            """)
        
        # ====================================================================
        # TAB 2: ENTRENAMIENTO
        # ====================================================================
        
        with gr.Tab("🎓 Entrenamiento"):
            gr.Markdown("""
            ### Entrenar modelo Bi-LSTM + Attention
            """)
            
            with gr.Row():
                epochs_input = gr.Slider(10, 200, 100, step=10, label="Épocas")
                batch_size_input = gr.Slider(8, 64, 16, step=8, label="Batch Size")
            
            augmentation_check = gr.Checkbox(
                label="Data Augmentation (Recomendado)",
                value=True
            )
            
            train_btn = gr.Button("🚀 Entrenar", variant="primary")
            train_output = gr.Textbox(label="Resultados", lines=15)
            
            train_btn.click(
                fn=train_model,
                inputs=[epochs_input, batch_size_input, augmentation_check],
                outputs=train_output
            )
            
            if not TRAINING_AVAILABLE:
                gr.Markdown("""
                ⚠️ **Módulos de entrenamiento no disponibles**
                
                Para habilitar entrenamiento, agrega a tu repositorio:
                - `src/advanced_lstm_model.py`
                - `src/training.py`
                
                Código completo en: `ALL_FILES_COMPLETE_CODE.md`
                """)
        
        # ====================================================================
        # TAB 3: TRADUCCIÓN
        # ====================================================================
        
        with gr.Tab("🔮 Traducción"):
            gr.Markdown("""
            ### Traducción en tiempo real
            
            1. Selecciona modelo entrenado
            2. Click "Cargar Modelo"
            3. Muestra tu seña a la webcam
            4. Observa la predicción
            """)
            
            model_selector = gr.Dropdown(
                label="Modelo",
                choices=get_model_list(),
                interactive=True
            )
            
            load_btn = gr.Button("📥 Cargar Modelo")
            load_status = gr.Textbox(label="Estado", lines=2)
            
            with gr.Row():
                with gr.Column(scale=1):
                    translate_webcam = gr.Image(sources=["webcam"], streaming=True, label="Webcam")
                
                with gr.Column(scale=1):
                    translate_output_img = gr.Image(label="Procesado")
                    translate_result = gr.Textbox(label="Predicción", lines=8)
            
            load_btn.click(
                fn=load_model_for_translation,
                inputs=model_selector,
                outputs=load_status
            )
            
            translate_webcam.stream(
                fn=translate_sign,
                inputs=translate_webcam,
                outputs=[translate_output_img, translate_result],
                stream_every=0.2
            )
            
            demo.load(
                fn=lambda: gr.Dropdown(choices=get_model_list()),
                outputs=model_selector
            )
        
        # ====================================================================
        # TAB 4: DATASET INFO
        # ====================================================================
        
        with gr.Tab("📊 Dataset"):
            gr.Markdown("### Información del Dataset")
            
            refresh_btn = gr.Button("🔄 Actualizar")
            dataset_info = gr.Textbox(label="Dataset", lines=20)
            
            refresh_btn.click(fn=show_dataset_info, outputs=dataset_info)
            demo.load(fn=show_dataset_info, outputs=dataset_info)
    
    gr.Markdown("""
    ---
    ### 📚 Información
    
    **💾 Datos**: `/content/drive/MyDrive/SignLanguageTranslator/`
    
    **🎥 Captura**: Webcam integrada en Gradio • Auto-pausa • Detección de manos
    
    **🎓 Entrenamiento**: Bi-LSTM + Attention • Data Augmentation • GPU
    
    **🔮 Traducción**: Tiempo real • Top-3 predicciones • Confianza
    """)

print("\n🎨 Iniciando interfaz completa...\n")
print("✅ Funcionalidades:")
print("   1. Captura con webcam en Gradio")
print("   2. Entrenamiento", "✅" if TRAINING_AVAILABLE else "⚠️ (requiere archivos)")
print("   3. Traducción en tiempo real")
print("   4. Info de dataset\n")

demo.launch(debug=True, share=True)

---
# 📝 Notas

## ✅ Mejoras Implementadas

### 1. Webcam en Gradio
- `gr.Image(sources=["webcam"], streaming=True)`
- No necesita JavaScript de Colab
- Se ve directamente en la interfaz

### 2. Captura Completa
- Preview en vivo
- Captura múltiple automática
- Detección de manos con auto-pausa
- Feedback en tiempo real

### 3. Entrenamiento Funcional
- Carga datos del dataset
- Data augmentation
- Entrena Bi-LSTM + Attention
- Guarda mejor modelo

### 4. Traducción en Tiempo Real
- Carga modelos entrenados
- Traduce mientras muestras señas
- Top-3 predicciones
- Muestra confianza

## 🎯 Flujo Completo

```
1. CAPTURA (Tab 1)
   ├─ Click "Preview" → Ver mano + keypoints
   ├─ Ingresar nombre: "hola"
   ├─ Muestras: 15
   ├─ Click "Iniciar Captura"
   ├─ Mostrar mano → Graba automáticamente
   ├─ Sin mano → Auto-pausa
   └─ Repetir para 5+ señas

2. ENTRENAMIENTO (Tab 2)
   ├─ Épocas: 100
   ├─ Batch: 16
   ├─ Augmentation: ✅
   ├─ Click "Entrenar"
   └─ Esperar ~10-15 min

3. TRADUCCIÓN (Tab 3)
   ├─ Seleccionar modelo
   ├─ Click "Cargar Modelo"
   ├─ Mostrar seña a webcam
   └─ Ver predicción en tiempo real

4. DATASET (Tab 4)
   └─ Ver estadísticas y progreso
```

## ⚠️ Importante

**Para entrenamiento completo necesitas:**
- `src/advanced_lstm_model.py`
- `src/training.py`

**Código en**: `ALL_FILES_COMPLETE_CODE.md`

**Si no los tienes**, la captura y traducción funcionarán pero entrenamiento mostrará mensaje de error.

---

**¡Disfruta tu Sign Language Translator completo! 🤟**